<a href="https://colab.research.google.com/github/mraarabzadeh/University_prj/blob/master/start3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [2]:
df = pd.read_csv('F:\\ut\\8\\proje\\data\\newbit.csv').dropna()
df = df.iloc[[x for x in range(0,20000,10)]]

In [3]:
class Bitcoin(nn.Module):
    def __init__(self,inSize, hiddenSize, outSize,ws):
        super().__init__()
        self.hiddenSize = hiddenSize
        self.inSize = inSize
        lstm = []
        for i,item in enumerate(hiddenSize):
            lstm.append(nn.LSTM(inSize,hiddenSize[i],num_layers=2,dropout=.2))
            inSize = hiddenSize[i]
        self.lstm = nn.ModuleList(lstm)
        self.linear = nn.Linear(hiddenSize[-1], outSize)
        self.ws = ws
        self.reset()
    def reset(self):
        self.hiddenMemory =[]
        for item in self.hiddenSize:
            self.hiddenMemory.append((torch.zeros(2,1,item).cuda(),\
                            torch.zeros(2,1,item).cuda()))
    def forward(self,seq):
        seq=seq.view(self.ws,-1,self.inSize)
        for i,item1 in enumerate(self.hiddenMemory):
            seq ,self.hiddenMemory[i]= self.lstm[i](seq, self.hiddenMemory[i])
        x = self.linear(seq)
        return x[-1]

In [4]:
def make_input(seq,ws):
    output = []
    for item in range(len(seq) - ws):
        output.append( ((seq[item:item+ws].reshape(1,-1)[0].cuda()),\
                        torch.FloatTensor(np.array(seq[item + ws:item + ws+1][-1][-1])).cuda()) )
    return output



In [5]:
WS = 5
instance = Bitcoin(1,[200,300,100],1,WS).cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(instance.parameters(), lr = .01)
torch.manual_seed(33)

In [17]:
traincol = ['Open','High','Low', 'Close', 'Volume_(Currency)', 'Weighted_Price']
resultCol = ['Close']

In [50]:
scaler = MinMaxScaler(feature_range=(-20, 20 ))
train_set = np.array(df[resultCol][-500:-100].values.astype(float)).reshape(-1,1)
train_set = scaler.fit_transform(train_set)
train_set = torch.FloatTensor(train_set)
train_set = make_input(train_set, WS)
test = df[resultCol][-230:].values.astype(float).reshape(-1,1)
test = scaler.fit_transform(test)
test = torch.FloatTensor(test)
test = make_input(test,WS)

In [ ]:
epoch = 200
optimizer = torch.optim.Adam(instance.parameters(), lr=.0001)
for i in range(epoch):
    for seq, y in train_set:
        instance.reset()
        pred = instance.forward(seq)
        loss = criterion(pred,y)
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
    print(i, loss)
#     if loss <= .001:
#       break

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0 tensor(1.5175, device='cuda:0', grad_fn=<MseLossBackward>)
1 tensor(0.0624, device='cuda:0', grad_fn=<MseLossBackward>)
2 tensor(0.0082, device='cuda:0', grad_fn=<MseLossBackward>)
3 tensor(0.2243, device='cuda:0', grad_fn=<MseLossBackward>)
4 tensor(0.1146, device='cuda:0', grad_fn=<MseLossBackward>)
5 tensor(0.5038, device='cuda:0', grad_fn=<MseLossBackward>)
6 tensor(0.0031, device='cuda:0', grad_fn=<MseLossBackward>)
7 tensor(3.8616e-05, device='cuda:0', grad_fn=<MseLossBackward>)
8 tensor(0.0621, device='cuda:0', grad_fn=<MseLossBackward>)
9 tensor(0.1326, device='cuda:0', grad_fn=<MseLossBackward>)
10 tensor(0.3077, device='cuda:0', grad_fn=<MseLossBackward>)
11 tensor(0.0003, device='cuda:0', grad_fn=<MseLossBackward>)
12 tensor(0.0035, device='cuda:0', grad_fn=<MseLossBackward>)
13 tensor(0.4151, device='cuda:0', grad_fn=<MseLossBackward>)
14 tensor(2.1963, device='cuda:0', grad_fn=<MseLossBackward>)
15 tensor(0.0311, device='cuda:0', grad_fn=<MseLossBackward>)
16 tensor(0.00

In [0]:
torch.save(instance.state_dict(),'/content/drive/My Drive/Colab Notebooks/start400_5_3L_R.pn')

In [43]:
instance = Bitcoin(1,[200,300,100],1,WS)
instance.load_state_dict(torch.load('F:\\ut\\8\\proje\\PN_result\\start400_5_3L_R.pn'))
instance = instance.cuda()

In [ ]:
# instance.eval()
preds = []
real = []
with torch.no_grad():
    for item, y in test:
        instance.reset()

        preds.append(instance.forward(item))
        real.append(y)


In [ ]:
plt.plot(range(220), preds[-220:],)
# plt.show()
plt.grid(color='r', linestyle='-', linewidth=.1, zorder=.1)
# plt.show()
plt.plot(range(220), real[-220:])
plt.show()

In [38]:
for key ,val in torch.load('F:\\ut\\8\\proje\\PN_result\\start400_5_3L_C.pn').items():
    print (key ,len(val))

lstm.0.weight_ih_l0 1200
lstm.0.weight_hh_l0 1200
lstm.0.bias_ih_l0 1200
lstm.0.bias_hh_l0 1200
lstm.0.weight_ih_l1 1200
lstm.0.weight_hh_l1 1200
lstm.0.bias_ih_l1 1200
lstm.0.bias_hh_l1 1200
lstm.1.weight_ih_l0 800
lstm.1.weight_hh_l0 800
lstm.1.bias_ih_l0 800
lstm.1.bias_hh_l0 800
lstm.1.weight_ih_l1 800
lstm.1.weight_hh_l1 800
lstm.1.bias_ih_l1 800
lstm.1.bias_hh_l1 800
lstm.2.weight_ih_l0 400
lstm.2.weight_hh_l0 400
lstm.2.bias_ih_l0 400
lstm.2.bias_hh_l0 400
lstm.2.weight_ih_l1 400
lstm.2.weight_hh_l1 400
lstm.2.bias_ih_l1 400
lstm.2.bias_hh_l1 400
linear.weight 1
linear.bias 1
